# Value Investing # 

_Value Investing_ is the strategy of analyzing stocks by their valuation and performance ratios, and investing in the stocks that are cheapest relative to these common measures. 

For this project, we are going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio.

***
## Library Imports

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

import yfinance as yf
# for the fundamental ratios it is easier to use yahoo_fin
import yahoo_fin.stock_info as si

import scipy.stats as stats 
import math

# this notebook contains future warnings which i wanted to be ignored
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Can read in previously saved data here:

In [2]:
# just P/E ratio data
value_df = pd.read_csv("PE_Ratio.csv", index_col=0)

# all value data
value_df = pd.read_csv("Value_Dataframe.csv", index_col=0)

Can use AlphaVantage API for up-to-date market data:

In [11]:
import requests
from secrets import ALPHAVANTAGE_API_KEY

## this endpoint extracts the basic financials of the stock
symbol = "GOOGL"
api_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHAVANTAGE_API_KEY}'
data = requests.get(api_url).json()

## Importing our list of stocks

In [2]:
tickers = si.tickers_sp500()
tickers

['A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF-B',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMY',
 'BR',
 'BRK-B',
 'BRO',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDAY',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COST',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTLT',
 'CTRA',
 'C

***
## Getting Market Data

We now pull the required market data from yahoo_fin package. 

### Building the data frame ###

We build the data frame and loop through our tickers to extract the trailing P/E ratio, as well as the price.

In [3]:
## THIS BLOCK TAKES APPROX 40 MINS TO RUN
# Note that yahoo_fin uses .append method from pandas, which may be 
# depreciated one day...the warning has been suppressed here

my_columns = ["Ticker", "Price", "P/E Ratio", "NumberSharesToBuy"]
value_df = pd.DataFrame(columns=my_columns)

# initialise parameters for progress bar
i = 0
n = len(tickers)

for tick in tickers:
    ## Timer if wanted
    i += 1
    p = np.round((100*i)/n)
    print(f"{p}% completed,", tick)

    # try getting the required data
    try:
        stock = si.get_quote_table(tick)
        price = stock["Quote Price"]
        pe_ratio = stock["PE Ratio (TTM)"]
        
        # build row to add to the dataframe
        row = pd.Series([tick, price, pe_ratio, 'N/A'], index = my_columns)
        value_df = pd.concat([value_df, row.to_frame().T], ignore_index=True)
    except:
        print("Error with: ", tick)

value_df.to_csv("PE_Ratio.csv")

0.0% completed, A
0.0% completed, AAL
1.0% completed, AAP
1.0% completed, AAPL
1.0% completed, ABBV
1.0% completed, ABC
1.0% completed, ABT
Error with:  ABT
2.0% completed, ACGL
2.0% completed, ACN
2.0% completed, ADBE
2.0% completed, ADI
2.0% completed, ADM


### Removing Glamour Stocks 

The opposite of a value stock is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the dataframe. 

We sort the dataframe by the P/E ratio and drop all stocks outside the top 50. 

In [4]:
## Best value means the smallest P/E ratio
value_df.sort_values("P/E Ratio", ascending=True, inplace=True)

## if some ratios were negative, could remove them here

top_50_pe = value_df[:50]
top_50_pe.reset_index(inplace=True, drop=True)

# view the new dataframe
top_50_pe

,Ticker,Price,P/E Ratio,NumberSharesToBuy
0,GOOGL,96.985001,1.29,NaN
1,GOOG,98.035004,1.31,NaN
2,MHK,115.900002,2.32,NaN
3,AIG,63.200100,3.75,NaN
4,APA,44.270000,4.19,NaN
5,CTRA,24.770000,4.50,NaN
6,DISH,13.985000,4.65,NaN
7,OGN,29.559999,4.95,NaN
8,CF,82.680000,5.40,NaN
9,DHI,95.440002,5.93,NaN


### Calculating the number of shares to buy

We define useful functions to extract portfolio size and then calculate the number of shares to buy, before adding it to the column of the dataframe.

In [11]:
def portfolio_size_input():
    # Get portfolio size
    portfolio_size = input("Enter the value of your portfolio:")
    try: 
        portfolio_size = float(portfolio_size)
    except ValueError:
        print("Portfolio Size is not a valid number.\n")

    return portfolio_size


def number_shares_to_buy(portfolio_size, portfolio):
    value = portfolio_size/len(portfolio.index)
    # loop through top_50 and place number of shares
    for i in range(0,len(portfolio)) :
        portfolio.loc[i,"NumberSharesToBuy"] = math.floor(value/portfolio.loc[i,"Price"])

In [6]:
portfolio_size = portfolio_size_input()

# add to the datafram column
number_shares_to_buy(portfolio_size, top_50_pe)

# view the updated dataframe
top_50_pe

,Ticker,Price,P/E Ratio,NumberSharesToBuy
0,GOOGL,96.985001,1.29,206.0
1,GOOG,98.035004,1.31,204.0
2,MHK,115.900002,2.32,172.0
3,AIG,63.200100,3.75,316.0
4,APA,44.270000,4.19,451.0
5,CTRA,24.770000,4.50,807.0
6,DISH,13.985000,4.65,1430.0
7,OGN,29.559999,4.95,676.0
8,CF,82.680000,5.40,241.0
9,DHI,95.440002,5.93,209.0


***
## Building a Better (and more realistic) Value Investment Strategy

Every valuation metric has certain flaws. For example, the P/E ratio doesn't work well with stocks that have negative earnings, like tech start-ups. 

Similarly, stocks that buyback their own shares are difficult to value using the P/B ratio (price-to-book). 

Investors typically use a `composite` basket of valuation metrics to build a robust quantitative value strategy.

In this next section, we will filter stocks with the lowest percentiles on the following metrics:

- P/E ratio
- P/B ratio
- Price-to-sales ratio
- EV-to-EBITDA (Enterprise Value to Earnings Before Interest, Tax, Depreciations and Amortizations)
- EV-to-Gross Profit

Not all of these ratios are provided by the API and instead will require computation once we have pulled the raw data.


In [3]:
my_columns = [
    "Ticker",
    "Price",
    "P/E Ratio",
    "P/E Percentile",
    "P/B Ratio",
    "P/B Percentile",
    "P/Sales Ratio",
    "P/Sales Percentile",
    "EV/EBITDA Ratio",
    "EV/EBITDA Percentile",
    "EV/Gross Ratio",
    "EV/Gross Percentile",
    "NumberSharesToBuy"
]

value_df = pd.DataFrame(columns=my_columns)

Now we can get market data to complete the data frame. 

In [4]:
## THIS BLOCK TAKES APPROX 80 MINS TO RUN
## Data can be loaded above
i = 0
n = len(tickers)

for tick in tickers:
    i += 1
    p = np.round((100*i)/n)
    print(f"{p}% completed,", tick)
    try:
        stock = si.get_stats_valuation(tick).iloc[:,0:2]
        price = si.get_quote_table(tick)["Quote Price"]
        pe_ratio = float(stock.iloc[2,1])
        pb_ratio = float(stock.iloc[6,1])
        ps_ratio = float(stock.iloc[5,1])
        ev_ebitda = float(stock.iloc[8,1])
        ev_gross = float(stock.iloc[7,1])

        row = pd.Series([tick, price, pe_ratio, 'N/A', 
                        pb_ratio, "N/A", ps_ratio, "N/A",
                        ev_ebitda, "N/A", ev_gross, "N/A", "N/A"], index = my_columns)
        value_df = pd.concat([value_df, row.to_frame().T], ignore_index=True)
    except:
        print("Error with: ", tick)

value_df.to_csv("Value_Dataframe.csv")

0.0% completed, A
0.0% completed, AAL
1.0% completed, AAP
1.0% completed, AAPL
1.0% completed, ABBV
1.0% completed, ABC
1.0% completed, ABT
2.0% completed, ACGL
2.0% completed, ACN
2.0% completed, ADBE
2.0% completed, ADI
2.0% completed, ADM
3.0% completed, ADP
3.0% completed, ADSK
3.0% completed, AEE
3.0% completed, AEP
3.0% completed, AES
4.0% completed, AFL
4.0% completed, AIG
4.0% completed, AIZ
4.0% completed, AJG
4.0% completed, AKAM
5.0% completed, ALB
5.0% completed, ALGN
5.0% completed, ALK
5.0% completed, ALL
5.0% completed, ALLE
6.0% completed, AMAT
6.0% completed, AMCR
6.0% completed, AMD
6.0% completed, AME
6.0% completed, AMGN
7.0% completed, AMP
7.0% completed, AMT
7.0% completed, AMZN
7.0% completed, ANET
7.0% completed, ANSS
8.0% completed, AON
8.0% completed, AOS
8.0% completed, APA
8.0% completed, APD
8.0% completed, APH
9.0% completed, APTV
9.0% completed, ARE
9.0% completed, ATO
9.0% completed, ATVI
9.0% completed, AVB
10.0% completed, AVGO
10.0% completed, AVY
10.

### Dealing with Missing Data

If the dataframe contains some missing data because all of the metrics we require are not available through the API, then we can use the pandas `isnull` method to identify data. 

Dealing with missing data is an important topic of data science. There are two main approaches:

- Drop missing data from the data set (pandas' `dropna` method is useful here). 
- Replace missing data with a new value (pandas `fillna` method is useful here). 

In this project, we will replace any missing data with the average non`NaN` data point from that column. Here's the code to do this:

In [5]:
value_df[value_df.isnull().any(axis=1)]
## Return a modified version of dataframe with rows that contain missing data

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/Sales Ratio,P/Sales Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Ratio,EV/Gross Percentile,NumberSharesToBuy
1,AAL,16.91,2.81,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
7,ACGL,64.074997,19.66,N/A,2.07,N/A,2.61,N/A,NaN,N/A,2.86,N/A,N/A
16,AES,26.014999,NaN,N/A,6.89,N/A,1.45,N/A,29.46,N/A,3.13,N/A,N/A
17,AFL,70.625,10.46,N/A,1.9,N/A,2.25,N/A,NaN,N/A,2.32,N/A,N/A
18,AIG,60.73,3.63,N/A,1.17,N/A,0.84,N/A,NaN,N/A,1.25,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,131.419998,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
495,ZBH,130.759995,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
496,ZBRA,325.540009,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
497,ZION,53.34,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A


In [6]:
for column in ["P/E Ratio","P/B Ratio", "P/Sales Ratio","EV/EBITDA Ratio","EV/Gross Ratio"]:
    value_df[column].fillna(value_df[column].mean(), inplace=True)

value_df

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/Sales Ratio,P/Sales Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Ratio,EV/Gross Percentile,NumberSharesToBuy
0,A,154.690002,37.190000,N/A,8.680000,N/A,6.810000,N/A,25.060000,N/A,6.970000,N/A,N/A
1,AAL,16.91,2.810000,N/A,11.559886,N/A,3.905054,N/A,13.017108,N/A,4.541778,N/A,N/A
2,AAP,149.059998,20.110000,N/A,3.420000,N/A,0.870000,N/A,13.350000,N/A,1.140000,N/A,N/A
3,AAPL,151.759995,25.610000,N/A,42.100000,N/A,6.300000,N/A,19.210000,N/A,6.320000,N/A,N/A
4,ABBV,144.550003,19.310000,N/A,16.020000,N/A,4.450000,N/A,12.400000,N/A,5.430000,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,131.419998,36.166136,N/A,11.559886,N/A,3.905054,N/A,13.017108,N/A,4.541778,N/A,N/A
495,ZBH,130.759995,36.166136,N/A,11.559886,N/A,3.905054,N/A,13.017108,N/A,4.541778,N/A,N/A
496,ZBRA,325.540009,36.166136,N/A,11.559886,N/A,3.905054,N/A,13.017108,N/A,4.541778,N/A,N/A
497,ZION,53.34,36.166136,N/A,11.559886,N/A,3.905054,N/A,13.017108,N/A,4.541778,N/A,N/A


### Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in our universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

- P/E
- P/B
- P/Sales
- EV/EBITDA
- EV/Gross

Here's how we'll do this:

In [7]:
metrics = {
    "P/E Ratio" : "P/E Percentile",
    "P/B Ratio" : "P/B Percentile",
    "P/Sales Ratio" : "P/Sales Percentile",
    "EV/EBITDA Ratio" : "EV/EBITDA Percentile",
    "EV/Gross Ratio" : "EV/Gross Percentile"
}

for metric in metrics.keys():
    for row in value_df.index:
        value_df.loc[row, metrics[metric]] = stats.percentileofscore(value_df[metric], value_df.loc[row, metric])

value_df

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/Sales Ratio,P/Sales Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Ratio,EV/Gross Percentile,NumberSharesToBuy
0,A,154.690002,37.190000,96.793587,8.680000,14.629259,6.810000,97.194389,25.060000,96.993988,6.970000,96.993988,N/A
1,AAL,16.91,2.810000,0.200401,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A
2,AAP,149.059998,20.110000,6.813627,3.420000,8.617234,0.870000,2.60521,13.350000,90.380762,1.140000,2.50501,N/A
3,AAPL,151.759995,25.610000,10.621242,42.100000,99.599198,6.300000,96.993988,19.210000,94.589178,6.320000,96.593186,N/A
4,ABBV,144.550003,19.310000,6.012024,16.020000,98.396794,4.450000,93.787575,12.400000,5.410822,5.430000,93.587174,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,131.419998,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A
495,ZBH,130.759995,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A
496,ZBRA,325.540009,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A
497,ZION,53.34,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A


### Calculating the RV Score

We'll now calculate the RV score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in our universe. 

The RV score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section. 

To calculate the arithmetic mean, we will use the `mean` function from Python's built in stats module. 

In [8]:
from statistics import mean

rv_score = []
for row in value_df.index:
    percentiles = []
    for metric in metrics.keys():
        percentiles.append(value_df.loc[row, metrics[metric]])
    
    ## now add the mean to the rv_score list
    rv_score.append(mean(percentiles))

value_df["RV Score"] = rv_score
value_df

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/Sales Ratio,P/Sales Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Ratio,EV/Gross Percentile,NumberSharesToBuy,RV Score
0,A,154.690002,37.190000,96.793587,8.680000,14.629259,6.810000,97.194389,25.060000,96.993988,6.970000,96.993988,N/A,80.521042
1,AAL,16.91,2.810000,0.200401,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A,41.563126
2,AAP,149.059998,20.110000,6.813627,3.420000,8.617234,0.870000,2.60521,13.350000,90.380762,1.140000,2.50501,N/A,22.184369
3,AAPL,151.759995,25.610000,10.621242,42.100000,99.599198,6.300000,96.993988,19.210000,94.589178,6.320000,96.593186,N/A,79.679359
4,ABBV,144.550003,19.310000,6.012024,16.020000,98.396794,4.450000,93.787575,12.400000,5.410822,5.430000,93.587174,N/A,59.438878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,131.419998,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A,52.384770
495,ZBH,130.759995,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A,52.384770
496,ZBRA,325.540009,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A,52.384770
497,ZION,53.34,36.166136,54.308617,11.559886,56.513026,3.905054,51.603206,13.017108,47.995992,4.541778,51.503006,N/A,52.384770


### Selecting the 50 Best Value Stocks

We can select the 50 best value stocks as we have done before:

In [9]:
value_df.sort_values("RV Score", ascending=True, inplace=True)
value_df_50 = value_df[:50]
value_df_50.reset_index(inplace=True, drop=True)

value_df_50

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/Sales Ratio,P/Sales Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Ratio,EV/Gross Percentile,NumberSharesToBuy,RV Score
0,DISH,13.975,4.960000,1.202405,0.470000,0.200401,0.560000,1.402806,9.110000,2.40481,1.720000,3.206413,N/A,1.683367
1,DHI,97.364998,6.390000,1.402806,1.750000,3.106212,1.070000,2.805611,5.200000,1.402806,1.140000,2.50501,N/A,2.244489
2,BWA,46.452999,14.010000,3.406814,1.670000,2.304609,0.750000,2.204409,7.320000,1.603206,0.940000,2.004008,N/A,2.304609
3,ADM,81.660004,13.370000,3.006012,1.950000,4.408818,0.510000,1.202405,10.240000,3.406814,0.630000,1.202405,N/A,2.645291
4,CNC,73.349998,21.970000,8.817635,1.680000,2.60521,0.310000,0.601202,9.730000,3.006012,0.330000,0.601202,N/A,3.126253
5,CTRA,24.16,4.490000,0.801603,1.540000,1.803607,2.220000,6.813627,3.140000,1.002004,2.380000,5.711423,N/A,3.226453
6,COP,111.099998,7.640000,2.004008,2.840000,7.41483,1.810000,5.210421,3.900000,1.202405,1.830000,3.607214,N/A,3.887776
7,BBY,86.684998,13.420000,3.206413,6.630000,12.825651,0.430000,1.002004,7.880000,1.803607,0.480000,0.801603,N/A,3.927856
8,CZR,53.549999,16.590000,5.01002,3.000000,7.815631,1.080000,3.006012,9.520000,2.805611,2.220000,4.809619,N/A,4.689379
9,BEN,31.450001,16.330000,4.609218,1.370000,1.002004,1.950000,5.811623,12.650000,6.012024,2.810000,7.014028,N/A,4.889780


### Caluulating the Number of Shares to Buy

We can use our prebuilt functions:

In [12]:
portfolio_size = portfolio_size_input()

number_shares_to_buy(portfolio_size, value_df_50)

value_df_50

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/Sales Ratio,P/Sales Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Ratio,EV/Gross Percentile,NumberSharesToBuy,RV Score
0,DISH,13.975,4.960000,1.202405,0.470000,0.200401,0.560000,1.402806,9.110000,2.40481,1.720000,3.206413,1431,1.683367
1,DHI,97.364998,6.390000,1.402806,1.750000,3.106212,1.070000,2.805611,5.200000,1.402806,1.140000,2.50501,205,2.244489
2,BWA,46.452999,14.010000,3.406814,1.670000,2.304609,0.750000,2.204409,7.320000,1.603206,0.940000,2.004008,430,2.304609
3,ADM,81.660004,13.370000,3.006012,1.950000,4.408818,0.510000,1.202405,10.240000,3.406814,0.630000,1.202405,244,2.645291
4,CNC,73.349998,21.970000,8.817635,1.680000,2.60521,0.310000,0.601202,9.730000,3.006012,0.330000,0.601202,272,3.126253
5,CTRA,24.16,4.490000,0.801603,1.540000,1.803607,2.220000,6.813627,3.140000,1.002004,2.380000,5.711423,827,3.226453
6,COP,111.099998,7.640000,2.004008,2.840000,7.41483,1.810000,5.210421,3.900000,1.202405,1.830000,3.607214,180,3.887776
7,BBY,86.684998,13.420000,3.206413,6.630000,12.825651,0.430000,1.002004,7.880000,1.803607,0.480000,0.801603,230,3.927856
8,CZR,53.549999,16.590000,5.01002,3.000000,7.815631,1.080000,3.006012,9.520000,2.805611,2.220000,4.809619,373,4.689379
9,BEN,31.450001,16.330000,4.609218,1.370000,1.002004,1.950000,5.811623,12.650000,6.012024,2.810000,7.014028,635,4.889780
